In [1]:
import numpy as np
import PIL, PIL.ImageDraw, PIL.ImageOps, PIL.ImageMath
import functools, itertools, io
import bitstring

from reedsolo import RSCodec, ReedSolomonError
from enum import Enum, IntEnum, auto

In [2]:
np.set_printoptions(edgeitems=30)
np.core.arrayprint._line_width = 180

In [3]:
def bch(n,k,_):
    """Return calculation function for (n,k,d)-BCH-Code."""
    def polynom_div(x, g):
        for i in range(k-1, -1, -1):
            if x & (1 << (n - k + i)):
                x ^= (g << i)
        return x
    return lambda x,g: (x << (n-k)) + polynom_div(x << (n-k), g)

In [4]:
class QR_Code:
    class Version(IntEnum):
        Version_2 = 2
        Version_3 = 3
        
    # Format Info
    mask_format_a = [[8,8,8,8,8,8,8,8,7,5,4,3,2,1,0], # y
                     [0,1,2,3,4,5,7,8,8,8,8,8,8,8,8]] # x
    
    mask_format_b = [[24,23,22,21,20,19,18, 8, 8, 8, 8, 8, 8, 8, 8], # y
                     [ 8, 8, 8, 8, 8, 8, 8,17,18,19,20,21,22,23,24]] # x
    
    class Mask(Enum):
        MASK_0 = 0b000, lambda y,x: (y + x) % 2 == 0,
        MASK_1 = 0b001, lambda y,x: y % 2 == 0,
        MASK_2 = 0b010, lambda y,x: x % 3 == 0,
        MASK_3 = 0b011, lambda y,x: (y + x) % 3 == 0,
        MASK_4 = 0b100, lambda y,x: (y // 2 + x // 3) % 2 == 0,
        MASK_5 = 0b101, lambda y,x: (y * x) % 2 + (y * x) % 3 == 0,
        MASK_6 = 0b110, lambda y,x: ((y * x) % 3 + y * x) % 2 == 0,
        MASK_7 = 0b111, lambda y,x: ((y * x) % 3 + y + x) % 2 == 0,


    class EC_Level(IntEnum):
        L = 0b01
        M = 0b00
        Q = 0b11
        H = 0b10

    # Info Table
    info = {
        Version.Version_2: {
            "size": (25,25),
            "total": 44,
            "capacity": {
                EC_Level.L: 34,
                EC_Level.M: 28,
                EC_Level.Q: 22,
                EC_Level.H: 16
            }
        },
    }
    
    # Patterns
    position_pattern = np.array([ \
        [1,1,1,1,1,1,1], \
        [1,0,0,0,0,0,1], \
        [1,0,1,1,1,0,1], \
        [1,0,1,1,1,0,1], \
        [1,0,1,1,1,0,1], \
        [1,0,0,0,0,0,1], \
        [1,1,1,1,1,1,1], \
    ], dtype=np.ubyte)
    alignment_pattern = np.array([
        [1,1,1,1,1],
        [1,0,0,0,1],
        [1,0,1,0,1],
        [1,0,0,0,1],
        [1,1,1,1,1]
    ], dtype=np.ubyte)
    
    def __init__(self, version: Version = Version.Version_2, ec_level: EC_Level = EC_Level.L, mask: Mask = Mask.MASK_0):
        self.version = version
        self.size = self.info[self.version]["size"]
        self.ec_level = ec_level
        self._mask = mask
        self._bch = bch(15,5,7)
        self.data = b""
        self._rsc = RSCodec(self.info[self.version]["total"] - self.capacity)
                
    def add_static_pattern(self):
        # Position Markers
        mask_pos_tl = np.ogrid[0:8, 0:8]
        mask_pos_tr = np.ogrid[0:8, 17:25]
        mask_pos_bl = np.ogrid[17:25, 0:8]
        self._bits[tuple(mask_pos_tl)] = np.pad(QR_Code.position_pattern, pad_width = ((0,1), (0,1)))
        self._bits[tuple(mask_pos_tr)] = np.pad(QR_Code.position_pattern, pad_width = ((0,1), (1,0)))
        self._bits[tuple(mask_pos_bl)] = np.pad(QR_Code.position_pattern, pad_width = ((1,0), (0,1)))
        # Alignment Markers
        # mask_align   = (16 <= x) & (x <= 20) & (16 <= y) & (y <= 20)
        mask_align = np.ogrid[16:21, 16:21]
        self._bits[tuple(mask_align)] = QR_Code.alignment_pattern
        # Timing Pattern
        mask_sync_h   = np.ogrid[6:7, 8:17]
        mask_sync_v   = np.ogrid[8:17, 6:7]
        self._bits[tuple(mask_sync_h)] = (mask_sync_h[1]+1) % 2
        self._bits[tuple(mask_sync_v)] = (mask_sync_v[0]+1) % 2
        # Dark Module
        mask_dark_module = self.dark_module()
        self._bits[mask_dark_module] = 1
    
    def dark_module(self):
        return ((4 * self.version) + 9, 8)
        
    @property
    def data(self):
        padded = itertools.chain(self._data, itertools.cycle(b"\xEC\x11"))
        bytelist = list(itertools.islice(padded, self.capacity))
        return bytes(bytelist)
    
    @data.setter
    def data(self, value: bytes):
        self._data = bytes(value)

    @property
    def capacity(self):
        return self.info[self.version]["capacity"][self.ec_level]
    
    @property
    def format_bits(self):
        mask_code, _ = self._mask.value
        format_info = (self.ec_level.value << 3) | mask_code
        format_bits = self._bch(format_info, 0b10100110111)
        return bitstring.pack("uint:15", format_bits ^ 0b101010000010010)
    
    @property
    def bits(self): 
        self._bits = np.full(self.size, 255, dtype=np.ubyte)
        self.add_static_pattern()
        
        self._bits[tuple(QR_Code.mask_format_a)] = self._bits[tuple(QR_Code.mask_format_b)] = self.format_bits
        
        self.data_mask = (self._bits == 255).astype(np.ubyte)
        bits = bitstring.BitArray(hex=self._rsc.encode(self.data).hex())
        # display(f"{bits=} len={len(bits)}")
        for pos, bit in zip(self.position_stream(), bits):
            self._bits[pos] = bit

        self._bits ^= (self.data_mask & self.mask).astype(np.ubyte)
            
        return self._bits

    def position_stream(self):
        y,x = self.size
        x_hat = x - 1
        while x_hat > 0:
            for ys in [range(y-1, 0-1, -1), range(0, y, 1)]:
                for y_hat in ys:
                    if self._bits[y_hat, x_hat] == 255:
                        yield (y_hat, x_hat)
                    if self._bits[y_hat, x_hat-1] == 255:
                        yield (y_hat, x_hat-1)
                x_hat -= 2
                # Special case: vertical timing pattern at x=6
                if x_hat == 6:
                    x_hat = x_hat - 1
    
    @property
    def mask(self):
        n,m = self.size
        y,x = np.mgrid[0:n, 0:m]
        _, mask_fn = self._mask.value
        return mask_fn(y,x).astype(np.ubyte)
        
    @staticmethod
    def draw(img=None, grid=True, scale=10):
        width, height = [coord * scale for coord in img.shape]
        img = PIL.Image.fromarray(255 * (1-img), mode="L")
        out = img.resize((width, height), resample=PIL.Image.NEAREST)

        if grid:
            grid_color = 200
            draw = PIL.ImageDraw.Draw(out)
            for x in range(0, width, scale):
                draw.line((x, 0) + (x, height), fill=grid_color)
            for y in range(0, height, scale):
                draw.line((0, y) + (width, y), fill=grid_color)  
        return out

In [5]:
import abc

class Encoder(abc.ABC):
    class EncodingMode(IntEnum):
        NUMERIC = 0b0001
        ALPHA   = 0b0010
        BYTE    = 0b0100
        KANJI   = 0b1000
        ECI     = 0b0111

    @abc.abstractmethod
    def __bytes__(self):
        pass
    
class ByteEncoder(Encoder):
    def __init__(self, byte_str, length = None):
        self._bytes = byte_str
        self._len   = len(byte_str)
        self._bitstring = bitstring.pack('uint:4, uint:8, hex', self.EncodingMode.BYTE, self._len, self._bytes.hex())
    
    def __len__(self):
        return len(self._bitstring)
    
    def __bytes__(self):
        return self._bitstring.tobytes()

In [6]:
qr = QR_Code(ec_level = QR_Code.EC_Level.M, mask = QR_Code.Mask.MASK_5)

In [7]:
# qr.data = ByteEncoder(b"nland{61mm3_5um_qr_fun}")
qr.data = ByteEncoder(b"DUMMYFLAG")

In [8]:
stage = 0
QR_Code.draw(qr.bits, scale=16, grid=False).save(f"output-{stage}.png")

In [9]:
img = qr.bits
for mask in QR_Code.Mask:
    # skip existing mask
    if mask == qr._mask:
        continue
    _, mask_fn = mask.value
    n,m = img.shape
    y,x = np.mgrid[0:n, 0:m]
    img ^= (qr.data_mask & mask_fn(y,x))

In [10]:
stage = 1
lower = QR_Code.draw(img, scale=16, grid=False)
lower.save(f"output-{stage}.png")

## Stage 2: Merge with upper layer

In [11]:
import codecs, png

In [12]:
upper = PIL.Image.open("Flag_of_FOTW.png")

In [13]:
size = max(lower.size, upper.size)

In [14]:
mode, colors = upper.palette.getdata()
n_colors = len(colors)//3
upper_arr = np.array(upper)

In [15]:
resized = PIL.Image.new("L", size, color=255)
resized.paste(lower, tuple(np.subtract(size, lower.size) // 2))

In [16]:
mask = np.array(resized) < 128
upper_arr[mask] += n_colors

In [17]:
output = PIL.Image.fromarray(upper_arr, mode="P")
palette = colors * 2
output.putpalette(palette)

In [18]:
stage = 2
output.save(f"output-{stage}.png")

## Additional distortions

### Stage 3: Add exif data

In [19]:
stage = 3
ifile = PIL.Image.open(f"output-{stage-1}.png")
exif = ifile.getexif()
# UserComment
exif[0x9286] = bytes(i % 0x15 for i in range(1024)) + codecs.encode("Forget steghide. This is no stego challenge.", "utf8") 
ifile.save(f"output-{stage}.png", exif=exif)

### Stage 4: Add compressed zTXt chunk

In [20]:
import gzip, struct

In [21]:
stage = 4
text = gzip.compress(codecs.encode("Seven masks too many!", "utf8"))
index = 2

In [22]:
reader = png.Reader(filename=f"output-{stage-1}.png")
chunks = reader.chunks()
chunk_list = list(chunks)
chunk_list.insert(index, (b"zTXt", b"hint" + b"\x00" + b"\x01" + text))
with open(f"output-{stage}.png", "wb") as ofile:
    png.write_chunks(ofile, chunk_list)

### Stage 5: Add tEXt chunk

In [23]:
lyrics = """We're no strangers to Exif
You know the rules and so do I
A full commitment's what I'm thinking of
You wouldn't get this from any other guy

I just wanna tell you how I'm feeling
Gotta make you understand

Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you"""
text = codecs.encode(b"Try jumping, amazon treasure ahead.", "base64")
stage = 5
index = 1

In [24]:
reader = png.Reader(filename=f"output-{stage-1}.png")
chunks = reader.chunks()
chunk_list = list(chunks)
chunk_list.insert(index, (b"tEXt", b"hint" + b"\0" + text))
chunk_list.insert(index, (b"tEXt", b"Comment" + b"\0" + codecs.encode(lyrics, "utf8")))
with open(f"output-{stage}.png", "wb") as ofile:
    png.write_chunks(ofile, chunk_list)

### Stage 6: Add hint after IEND chunk

In [25]:
stage = 6
with open(f"output-{stage-1}.png", "rb") as ifile:
    with open(f"output-{stage}.png", "wb") as ofile:
        data = ifile.read()
        banner = codecs.encode("No, seriously. This ain't no stego challenge!", "utf8")
        ofile.write(data + banner)